In [1]:
import pandas as pd
import json
import sys
import re
sys.path.append('..')

In [17]:
from llm_class import LLM

llm = LLM(model_id="bielik", key_path='../key.txt')

In [18]:
df = pd.read_csv("../data/dialog_witcher_dataset.csv", delimiter="|")
with open('../data/speakers_characteristics.json', 'r') as f:
    speaker_data = json.load(f) 

In [19]:
def extract_translation(llm_resposne):
    if llm_resposne is None:
        return None
    match = re.search(r'\[(.*?)\]', llm_resposne)
    if match:
        return match.group(1)
    return None

In [8]:
prompt_single_line = """
Oto kontekst: Wypowiedź mówiona jest przez {speaker} ({gender}).

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [9]:
prompt_multiple_lines = """ 
Oto kontekst: poniżej znajduje się kawałek {monolog_or_dialog}{speakers}

{dialog}

Na podstawie podanego {monolog_or_dialog} oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Na podstawie podanego {monolog_or_dialog} oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [10]:
prompt_multiple_linesv2 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: Dubhenn haern am glâdeal.
Tłumaczenie: [Dubhenn haern am glâdeal.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Oto kontekst: poniżej znajduje się kawałek {monolog_or_dialog}{speakers}

{dialog}

Na podstawie podanego {monolog_or_dialog} oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych: {text_to_translate}

"""

In [11]:
prompt_single_linev2 = """
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Oto kontekst: Wypowiedź mówiona jest przez {speaker} ({gender}).

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [20]:
few_shot_dialog = """ 
Poniżej znajduje się kawałek dialogu, rozmawia w nim Geralt (mężczyzna) z Elf Female 03 (kobieta)

1. Elf Female 03 - Easily recognized. I'm proud of it, never thought to conceal it in any way.
2. Geralt - From what I know, elves in Novigrad aren't exactly treated in a civil manner.
3. Elf Female 03 - That's putting it lightly.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie wypowiedź numer 2 z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Z tego co wiem, w Novigradzie elfy nie są zbyt dobrze traktowane.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Avallach z Cirilla (kobieta)

1. Avallach - The ice… it's a spell. You must find the mage who cast it, destroy his staff.
2. Cirilla - I'll not stop with his staff.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie wypowiedź numer 1 z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Ten lód... To zaklęcie. Musisz zniszczyć kostur czarodzieja, który je rzucił.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Ida Emean z Yennefer (kobieta) oraz z Geralt (mężczyzna)

1. Yennefer - Stop twisting my words.
2. Ida Emean - Cáed'mil, Yennefer aep Vengerberg. Gwynbleidd.
3. Geralt - Ceádmil, Aen Saevherne.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie wypowiedź numer 2 z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Cáed'mil, Yennefer aep Vengerberg. Gwynbleidd.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Geralt (mężczyzna) z Lugos The Glaucous

1. Geralt - If there's one thing I'm not afraid of, it's nekkers.
2. Lugos The Glaucous - Tis but the start. Real challenge awaits at the cave's end.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie wypowiedź numer 1 z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Czego jak czego, ale nekkerów to się akurat nie boję.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Geralt (mężczyzna) z Skellige Guard 04 (mężczyzna)

1. Skellige Guard 04 - Shame to let such a lad waste away behind bars… Come on. Maybe Lugos'll try you today.
2. Geralt - And they say violence is never the answer…
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie wypowiedź numer 2 z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [A mówią, że przemoc nigdy nie jest rozwiązaniem...]
"""

In [30]:
few_shot_dialogv2 = """ 
Poniżej znajduje się kawałek dialogu, rozmawia w nim Geralt (mężczyzna) z Elf Female 03 (kobieta)

Elf Female 03 - Easily recognized. I'm proud of it, never thought to conceal it in any way.
Geralt - From what I know, elves in Novigrad aren't exactly treated in a civil manner.
Elf Female 03 - That's putting it lightly.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie drugą wypowiedź z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Z tego co wiem, w Novigradzie elfy nie są zbyt dobrze traktowane.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Avallach z Cirilla (kobieta)

Avallach - The ice… it's a spell. You must find the mage who cast it, destroy his staff.
Cirilla - I'll not stop with his staff.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie pierwszą wypowiedź z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Ten lód... To zaklęcie. Musisz zniszczyć kostur czarodzieja, który je rzucił.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Ida Emean z Yennefer (kobieta) oraz z Geralt (mężczyzna)

Yennefer - Stop twisting my words.
Ida Emean - Cáed'mil, Yennefer aep Vengerberg. Gwynbleidd.
Geralt - Ceádmil, Aen Saevherne.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie drugą wypowiedź z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Cáed'mil, Yennefer aep Vengerberg. Gwynbleidd.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Geralt (mężczyzna) z Lugos The Glaucous

Geralt - If there's one thing I'm not afraid of, it's nekkers.
Lugos The Glaucous - Tis but the start. Real challenge awaits at the cave's end.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie pierwszą wypowiedź z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Czego jak czego, ale nekkerów to się akurat nie boję.]


Poniżej znajduje się kawałek dialogu, rozmawia w nim Geralt (mężczyzna) z Skellige Guard 04 (mężczyzna)

Skellige Guard 04 - Shame to let such a lad waste away behind bars… Come on. Maybe Lugos'll try you today.
Geralt - And they say violence is never the answer…
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie drugą wypowiedź z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [A mówią, że przemoc nigdy nie jest rozwiązaniem...]
"""

In [13]:
few_shot_single_line = """ 
Wypowiedź mówiona jest przez Geralt (mężczyzna)

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: I am a very good artist.
Przykładowe tłumaczenie: [Jestem bardzo dobrym artystą.]


Wypowiedź mówiona jest przez Yennefer (kobieta)

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: Dubhenn haern am glâdeal.
Przykładowe tłumaczenie: [Dubhenn haern am glâdeal.]


Wypowiedź mówiona jest przez Random person

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: Have you heard from her recently?
Przykładowe tłumaczenie: [Miałeś z nią ostatnio kontakt?]


Wypowiedź mówiona jest przez Troll

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: Soup is very yummy, I want more.
Przykładowe tłumaczenie: [Zupa być pyszna, chcieć więcej]
"""

In [ ]:
prompt_multiple_linesv3 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie wypowiedzi w dialogu:
{examples}


Oto kontekst: Poniżej znajduje się kawałek dialogu{speakers}

{dialog}

Na podstawie podanego dialogu, kontekstu oraz podanych wyżej przykładów, przetłumacz tylko i wyłącznie wypowiedź numer {numer} z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Napisz tylko i wyłącznie końcową wersje tłumaczenia w kwadratowych nawiasach z dialogu powyżej. Nie wypisuj nic więcej.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych:

"""

In [31]:
prompt_multiple_linesv4 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie wypowiedzi w dialogu:
{examples}


Oto kontekst: Poniżej znajduje się kawałek dialogu{speakers}

{dialog}

Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Napisz tylko i wyłącznie końcową wersje tłumaczenia w kwadratowych nawiasach z dialogu powyżej. Nie wypisuj nic więcej.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych:

"""

In [32]:
prompt_single_linev3 = """
Twoim zadaniem jest przetłumaczenie wypowiedzi na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
{examples}


Oto kontekst: Wypowiedź mówiona jest przez {speaker}{gender}.

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [14]:
prompt_default = """
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {en_text}
"""

In [61]:
def choose_and_format_prompt(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')

    #jezeli pojedyncza wypowiedz
    if speaker1 == None and speaker3 == None:
        return prompt_single_linev3.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate, examples=few_shot_single_line)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    monolog_or_dialog = ""
    speakers = ""
    dialog = ""
    first_or_second = ""

    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1

    def number_to_st(i):
        if i == 1:
            return "1."
        elif i == 2:
            return "2."
        else:
            return "3."
    i=1
    numer = 1

    #tworzenie dialogu
    if speaker1:
        dialog += f"{number_to_st(i)} {speaker1} - {prev_text}\n"
        numer = i
        i += 1

        #piersza czy druga wypowiedz
        first_or_second = "drugą"
    else:
        first_or_second = "pierwszą"

    if speaker2:
        dialog += f"{number_to_st(i)} {speaker2} - {text_to_translate}\n"
        numer = i
        i += 1
    if speaker3:
        dialog += f"{number_to_st(i)} {speaker3} - {next_text}\n"

    #handling monolog
    if is_monolog:
        monolog_or_dialog = "monologu"
        speakers = f" wypowiadany przez {speaker2}{speaker2_gender}"
    #handling dialog
    else:
        monolog_or_dialog = "dialogu"
        if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
            speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
        elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
            speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
        else:
            speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_linesv3.format(monolog_or_dialog=monolog_or_dialog, speakers=speakers, dialog=dialog, first_or_second=first_or_second, text_to_translate=text_to_translate, examples=few_shot_dialog, numer=numer)

In [51]:
df = df.where(pd.notna(df), None)

In [62]:
sampled_df = df.sample(n=10, random_state=532423).reset_index(drop=True)
sampled_df

,speaker,prev_text,prev_speaker,next_text,next_speaker,en_text,pl_text
0,Novigrad Old Woman 03,Watch where you step!,Novigrad Old Woman 03,Help!,Novigrad Old Woman 03,Save yourselves!,Ratujcie się!
1,Geralt,"As custom has it, the clan sword should go to ...",Cerys,Serious enough for Udalryk to break a sacred l...,Cerys,"Huh, serious insult in these lands.","Uuu, w waszych stronach to poważna zniewaga."
2,Nathanel,Yet again you disturb me. And I so dislike bei...,Nathanel,"Try it, you bastard.",Geralt,I was to play with Sweet Nettie. Render her no...,"Miałem się zabawić ze Słodką Anneke. Sprawić, ..."
3,Novigrad Rich Woman 03,"Do not, I repeat, do not come any closer.",Novigrad Rich Woman 03,How dare you?!,Novigrad Rich Woman 03,I read one look is all it takes to get scabs.,"W jednej księdze stalo, że od samego spojrzeni..."
4,Scoiatael Elf Man 34,I don't know…,Scoiatael Elf Man 34,Bloody weather.,Scoiatael Elf Man 34,Hey!,Uważaj!
5,Novigrad Noblewoman 02,"Ugh. Well, killer's already escaped. Lead the ...",Geralt,Murder!,Novigrad Noblewoman 02,Guards!,Straż!
6,Geralt,And I'll not lift it till they bring me worthy...,Fugas,The local peasant folk call me the Allgod.,Fugas,Who the hell are you supposed to be?,Coś ty za jeden?
7,Geralt,Vintages from 1255 to 1264…,Geralt,And... zilch.,Geralt,"Hm. Satisfying click… in the next room over, I...","Coś przeskoczyło... W pokoju obok, zdaje się."
8,Novigrad Boy 02,None,None,That's a perty necklace! Where'd you get it?,Novigrad Prostitute 02,"Mum, look what I found!","Zobacz co znalazłem, mama."
9,Marabella,None,None,None,None,Eternal Fire!,Na Wieczny Ogień!


In [63]:
for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    original_translation = row['pl_text']

    prompt = choose_and_format_prompt(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)

    print("Prompt:")
    print(prompt)
    print()
    print()

    response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)

    print("Response:")
    print(response)
    print()
    print()

    translation = extract_translation(response)

    if translation == None:
        print(":(((")
        continue

    print("Translation: ", translation)
    print("Original: ", original_translation)
    print()
    print()
    print("============================================")   
    print()
    print()
    

Prompt:
 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
 
Poniżej znajduje się kawałek dialogu, rozmawia w nim Geralt (mężczyzna) z Elf Female 03 (kobieta)

1. Elf Female 03 - Easily recognized. I'm proud of it, never thought to conceal it in any way.
2. Geralt - From what I know, elves in Novigrad aren't exactly treated in a civil manner.
3. Elf Female 03 - That's putting it lightly.
 
Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie wypowiedź numer 2 z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych.

Przykładowe tłumaczenie: [Z tego co wiem, w Novigradzie elfy nie są

In [64]:
sampled_df = df.sample(n=2000, random_state=444).reset_index(drop=True)
sampled_df

,speaker,prev_text,prev_speaker,next_text,next_speaker,en_text,pl_text
0,Lambert,"Ha, ha.",Lambert,What'd you expect? Scent of violets?,Geralt,"Oof, place reeks of mould…",Ale tu cuchnie grzybem.
1,Keira Metz,"A plague can go wrong, go wild. A sword won't....",Geralt,"See this? Bedbugs, Geralt. Bloody bedbugs! I, ...",Keira Metz,Shut up for a moment and look at me.,Zamknij się na moment i popatrz na mnie.
2,Geralt,Show me what you got.,Geralt,"Course I do. Vizima or Mahakam this ain't, but...",Dwarf Man 02,Greetings. Got anything in stock besides hoes ...,Witaj. Masz na sprzedaż coś poza podkowami i m...
3,Novigrad Refugee Woman 03,Oh no.,Novigrad Refugee Woman 03,"Eh, swine.",Novigrad Refugee Woman 03,Where're my young'uns?! Aaargh!,Gdzie moje dzieci?! Aaaa!
4,Geralt,As have I. Neither is to leave here alive.,Novigrad Rich Man 01,I paid in advance because I knew I'd recover m...,Novigrad Rich Man 01,"If these boys are here to help us kill rats, t...","Jeżeli przyprowadziłeś tych panów, żeby nam po..."
...,...,...,...,...,...,...,...
1995,Geralt,Aaaarrrggghhhh!,Udalryk,I saw… saw it sink into the ground.,Udalryk,The hym's gone! I can't see it!,Him zniknął! Nie widzę go!
1996,Cirilla,"A coincidence, really. I had no desire to hunt...",Cirilla,"I was wandering through the forest, breathing ...",Cirilla,I'd taken nothing remotely resembling a huntin...,"Nie wzięłam żadnej sensownej broni, bo nie mia..."
1997,Baron,You tell me. Man or monster? My men call him U...,Baron,Funny story? How so?,Geralt,"Funny story, actually. I won him in a game of ...","A widzisz, śmieszna historia: wygrałem go w ka..."
1998,Priscilla,Just one problem - no dopplers in that story. ...,Geralt,"Hm, all right. So it's a comedy?",Geralt,Nothing simpler. We need merely swap the doppl...,"Nie szkodzi, zamienimy Jeża na dopplera. Najwa..."


In [65]:
for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    original_translation = row['pl_text']

    prompt = choose_and_format_prompt(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)

    response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)

    translation = extract_translation(response)

    if translation == None:
        sampled_df.at[index, f'dialog_prototype'] = ""
        continue

    sampled_df.at[index, f'dialog_prototype'] = translation

    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/dialog_prototype_experimentsv3.csv", sep="|", index=False)

Processed 100 out of 2000 samples...
Processed 200 out of 2000 samples...
Processed 300 out of 2000 samples...
Processed 400 out of 2000 samples...
Processed 500 out of 2000 samples...
Processed 600 out of 2000 samples...
Processed 700 out of 2000 samples...
Processed 800 out of 2000 samples...
Processed 900 out of 2000 samples...
Processed 1000 out of 2000 samples...
Processed 1100 out of 2000 samples...
Processed 1200 out of 2000 samples...
Processed 1300 out of 2000 samples...
Processed 1400 out of 2000 samples...
Processed 1500 out of 2000 samples...
Processed 1600 out of 2000 samples...
Processed 1700 out of 2000 samples...
Processed 1800 out of 2000 samples...
Processed 1900 out of 2000 samples...
Processed 2000 out of 2000 samples...


In [ ]:
def choose_and_format_promptv2(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')
    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""


    #jezeli pojedyncza wypowiedz
    if is_monolog:
        return prompt_single_linev3.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate, examples=few_shot_single_line)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    speakers = ""
    dialog = ""
    first_or_second = ""

    

    def number_to_st(i):
        if i == 1:
            return "1."
        elif i == 2:
            return "2."
        else:
            return "3."
    i=1
    numer = 1

    #tworzenie dialogu
    if speaker1 and speaker1 != speaker2:
        dialog += f"{number_to_st(i)} {speaker1} - {prev_text}\n"
        numer = i
        i += 1

        #piersza czy druga wypowiedz
        first_or_second = "drugą"
    else:
        first_or_second = "pierwszą"

    if speaker2:
        dialog += f"{number_to_st(i)} {speaker2} - {text_to_translate}\n"
        numer = i
        i += 1
    if speaker3 and speaker3 != speaker2:
        dialog += f"{number_to_st(i)} {speaker3} - {next_text}\n"

  
    if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
    elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
    else:
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_linesv3.format(speakers=speakers, dialog=dialog, first_or_second=first_or_second, text_to_translate=text_to_translate, examples=few_shot_dialog, numer=numer)

In [32]:
def choose_and_format_promptv3(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')
    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""


    #jezeli pojedyncza wypowiedz
    if is_monolog:
        return prompt_single_linev3.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate, examples=few_shot_single_line)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    speakers = ""
    dialog = ""
    first_or_second = ""

    

    #tworzenie dialogu
    if speaker1 and speaker1 != speaker2:
        dialog += f"{speaker1} - {prev_text}\n"
 
        #piersza czy druga wypowiedz
        first_or_second = "drugą"
    else:
        first_or_second = "pierwszą"

    if speaker2:
        dialog += f"{speaker2} - {text_to_translate}\n"

    if speaker3 and speaker3 != speaker2:
        dialog += f"{speaker3} - {next_text}\n"

  
    if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
    elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
    else:
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_linesv4.format(speakers=speakers, dialog=dialog, first_or_second=first_or_second, text_to_translate=text_to_translate, examples=few_shot_dialogv2)

In [34]:
sampled_df = df.sample(n=2000, random_state=444).reset_index(drop=True)

for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    original_translation = row['pl_text']

    prompt1 = choose_and_format_promptv2(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)
    prompt2 = choose_and_format_promptv3(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)

    response1 = llm.prompt_chat_custom_temperature(prompt=prompt1, temperature=0.0)
    response2 = llm.prompt_chat_custom_temperature(prompt=prompt2, temperature=0.0)

    translation1 = extract_translation(response1)
    translation2 = extract_translation(response2)

    if translation1 == None:
        sampled_df.at[index, f'dialog_prototype1'] = ""
        
    if translation2 == None:
        sampled_df.at[index, f'dialog_prototype2'] = ""

    sampled_df.at[index, f'dialog_prototype1'] = translation1
    sampled_df.at[index, f'dialog_prototype2'] = translation2

    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/dialog_prototype_experimentsv4.csv", sep="|", index=False)

Processed 100 out of 2000 samples...
Processed 200 out of 2000 samples...
Processed 300 out of 2000 samples...
Processed 400 out of 2000 samples...
Processed 500 out of 2000 samples...
Processed 600 out of 2000 samples...
Processed 700 out of 2000 samples...
Processed 800 out of 2000 samples...
Processed 900 out of 2000 samples...
Processed 1000 out of 2000 samples...
Processed 1100 out of 2000 samples...
Processed 1200 out of 2000 samples...
Processed 1300 out of 2000 samples...
Processed 1400 out of 2000 samples...
Processed 1500 out of 2000 samples...
Processed 1600 out of 2000 samples...
Processed 1700 out of 2000 samples...
Processed 1800 out of 2000 samples...
Processed 1900 out of 2000 samples...
Processed 2000 out of 2000 samples...


In [35]:
for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    original_translation = row['pl_text']

    prompt = prompt_default.format(en_text=text_to_translate)
  
    response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)

    sampled_df.at[index, f'default'] = response

    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/dialog_prototype_experimentsv5.csv", sep="|", index=False)

Processed 100 out of 2000 samples...
Processed 200 out of 2000 samples...
Processed 300 out of 2000 samples...
Processed 400 out of 2000 samples...
Processed 500 out of 2000 samples...
Processed 600 out of 2000 samples...
Processed 700 out of 2000 samples...
Processed 800 out of 2000 samples...
Processed 900 out of 2000 samples...
Processed 1000 out of 2000 samples...
Processed 1100 out of 2000 samples...
Processed 1200 out of 2000 samples...
Processed 1300 out of 2000 samples...
Processed 1400 out of 2000 samples...
Processed 1500 out of 2000 samples...
Processed 1600 out of 2000 samples...
Processed 1700 out of 2000 samples...
Processed 1800 out of 2000 samples...
Processed 1900 out of 2000 samples...
Processed 2000 out of 2000 samples...


In [ ]:
prompt_multiple_linesv3 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]


Oto kontekst: Poniżej znajduje się kawałek dialogu{speakers} 

{dialog}

Na podstawie podanego dialogu, kontekstu oraz podanych wyżej przykładów, przetłumacz tylko i wyłącznie wypowiedź numer {numer} z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Napisz tylko i wyłącznie końcową wersje tłumaczenia w kwadratowych nawiasach z dialogu powyżej. Nie wypisuj nic więcej.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych:

"""

In [37]:
prompt_multiple_linesv4 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]

Oto kontekst: Poniżej znajduje się kawałek dialogu{speakers}

{dialog}

Na podstawie podanego dialogu oraz kontekstu, przetłumacz tylko i wyłącznie {first_or_second} wypowiedź z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Napisz tylko i wyłącznie końcową wersje tłumaczenia w kwadratowych nawiasach z dialogu powyżej. Nie wypisuj nic więcej.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych:

"""

In [34]:
def choose_and_format_promptv2(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')
    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""


    #jezeli pojedyncza wypowiedz
    if is_monolog:
        return prompt_single_linev3.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate, examples=few_shot_single_line)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    speakers = ""
    dialog = ""


    def number_to_st(i):
        if i == 1:
            return "1."
        elif i == 2:
            return "2."
        else:
            return "3."
    i=1
    numer = 1

    if speaker1 and speaker1 != speaker2:
        dialog += f"{number_to_st(i)} {speaker1} - {prev_text}\n"
        numer = i
        i += 1

    if speaker2:
        dialog += f"{number_to_st(i)} {speaker2} - {text_to_translate}\n"
        numer = i
        i += 1
    if speaker3 and speaker3 != speaker2:
        dialog += f"{number_to_st(i)} {speaker3} - {next_text}\n"

  
    if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
    elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
    else:
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_linesv3.format(speakers=speakers, dialog=dialog, text_to_translate=text_to_translate, numer=numer)

In [41]:
def choose_and_format_promptv3(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')
    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""


    #jezeli pojedyncza wypowiedz
    if is_monolog:
        return prompt_single_linev3.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate, examples=few_shot_single_line)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    speakers = ""
    dialog = ""
    first_or_second = ""

    

    #tworzenie dialogu
    if speaker1 and speaker1 != speaker2:
        dialog += f"{speaker1} - {prev_text}\n"
 
        #piersza czy druga wypowiedz
        first_or_second = "drugą"
    else:
        first_or_second = "pierwszą"

    if speaker2:
        dialog += f"{speaker2} - {text_to_translate}\n"

    if speaker3 and speaker3 != speaker2:
        dialog += f"{speaker3} - {next_text}\n"

  
    if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
    elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
    else:
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_linesv4.format(speakers=speakers, dialog=dialog, first_or_second=first_or_second, text_to_translate=text_to_translate)

In [45]:
sampled_df = df.sample(n=2000, random_state=444).reset_index(drop=True)

for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    original_translation = row['pl_text']

    prompt1 = choose_and_format_promptv2(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)
    prompt2 = choose_and_format_promptv3(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)

    response1 = llm.prompt_chat_custom_temperature(prompt=prompt1, temperature=0.0)
    response2 = llm.prompt_chat_custom_temperature(prompt=prompt2, temperature=0.0)

    translation1 = extract_translation(response1)
    translation2 = extract_translation(response2)

    if translation1 == None:
        sampled_df.at[index, f'dialog_prototype1'] = ""
        
    if translation2 == None:
        sampled_df.at[index, f'dialog_prototype2'] = ""

    sampled_df.at[index, f'dialog_prototype1'] = translation1
    sampled_df.at[index, f'dialog_prototype2'] = translation2


    prompt = prompt_default.format(en_text=text_to_translate)
  
    response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)

    sampled_df.at[index, f'default'] = response


    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/dialog_prototype_experimentsv5.csv", sep="|", index=False)

Processed 100 out of 2000 samples...
Processed 200 out of 2000 samples...
Processed 300 out of 2000 samples...
Processed 400 out of 2000 samples...
Processed 500 out of 2000 samples...
Processed 600 out of 2000 samples...
Processed 700 out of 2000 samples...
Processed 800 out of 2000 samples...
Processed 900 out of 2000 samples...
Processed 1000 out of 2000 samples...
Processed 1100 out of 2000 samples...
Processed 1200 out of 2000 samples...
Processed 1300 out of 2000 samples...
Processed 1400 out of 2000 samples...
Processed 1500 out of 2000 samples...
Processed 1600 out of 2000 samples...
Processed 1700 out of 2000 samples...
Processed 1800 out of 2000 samples...
Processed 1900 out of 2000 samples...
Processed 2000 out of 2000 samples...


In [5]:
import fasttext
model = fasttext.load_model("../lid.176.bin")

In [6]:
def is_english(text, regex):

    clean_text = str(text).replace('\n', ' ').strip()
    clean_text = str(clean_text).replace('…', ' ').strip()

    if not clean_text:
        return False
    
    if regex:
        clean_text = re.sub(r'[^a-zA-Z\s]', '', clean_text)

    if len(clean_text) < 15 :
        return True
    
    label, _ = model.predict(clean_text)
    language = label[0].replace('__label__', '')
    return language == 'en' 

In [10]:
prompt_multiple_linesv4 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]


Oto kontekst: Poniżej znajduje się kawałek dialogu{speakers}

{dialog}

Na podstawie podanego dialogu, kontekstu oraz podanych wyżej przykładów, przetłumacz tylko i wyłącznie wypowiedź numer {numer} z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
"""

In [25]:
prompt_multiple_linesv5 = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:

Przykładowe zdanie po angielsku: I am a very good artist.
Przykładowe tłumaczenie: [Jestem bardzo dobrym artystą.]

Przykładowe zdanie po angielsku: Have you heard from her recently?
Przykładowe tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Przykładowe zdanie po angielsku: I heard that Michael is going to be a father.
Przykładowe tłumaczenie: [Słyszałem, że Michael będzie ojcem.]


Oto kontekst: Poniżej znajduje się kawałek dialogu{speakers}

{dialog}

Na podstawie podanego dialogu i kontekstu, przetłumacz tylko i wyłącznie wypowiedź numer {numer} na język polski, oraz ostateczną wersje tłumaczenia zapisz w kwadratowych nawiasach, tak jak jest to pokazane w przykładowych tłumaczeniach.
"""

In [27]:
def choose_and_format_prompt(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')
    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""


    #jezeli pojedyncza wypowiedz
    if is_monolog:
        return prompt_single_linev3.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate, examples=few_shot_single_line)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    speakers = ""
    dialog = ""


    def number_to_st(i):
        if i == 1:
            return "1."
        elif i == 2:
            return "2."
        else:
            return "3."
    i=1
    numer = 1

    if speaker1 and speaker1 != speaker2:
        dialog += f"{number_to_st(i)} {speaker1} - {prev_text}\n"
        numer = i
        i += 1

    if speaker2:
        dialog += f"{number_to_st(i)} {speaker2} - {text_to_translate}\n"
        numer = i
        i += 1
    if speaker3 and speaker3 != speaker2:
        dialog += f"{number_to_st(i)} {speaker3} - {next_text}\n"

  
    if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
    elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
    else:
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_linesv5.format(speakers=speakers, dialog=dialog, text_to_translate=text_to_translate, numer=numer)

In [29]:
sampled_df = df.sample(n=2000, random_state=444).reset_index(drop=True)

for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    #czy po ang teskt
    if is_english(text_to_translate, False) == False and is_english(text_to_translate, True) == False:
        sampled_df.at[index, f'dialog_prototype'] = text_to_translate
        continue

    translation = None
    i = 0

    while translation == None and i < 3:
        prompt = choose_and_format_prompt(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)
        response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)
        translation = extract_translation(response)
        i+=1

    if translation == None:
        print(prompt)
        sampled_df.at[index, f'dialog_prototype'] = "ERROR"
        continue


    sampled_df.at[index, f'dialog_prototype'] = translation


    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/dialog_prototype_experimentsv7.csv", sep="|", index=False)

Processed 100 out of 2000 samples...
Processed 200 out of 2000 samples...
Processed 300 out of 2000 samples...
Processed 400 out of 2000 samples...
Processed 500 out of 2000 samples...
Processed 600 out of 2000 samples...
Processed 700 out of 2000 samples...
Processed 800 out of 2000 samples...
Processed 900 out of 2000 samples...
Processed 1000 out of 2000 samples...
 
Twoim zadaniem jest przetłumaczenie wypowiedzi na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:

Przykładowe zdanie po angielsku: I am a very good artist.
Przykładowe tłumaczenie: [Jestem bardzo dobrym artystą.]

Przykładowe zdanie po angielsku: Have you heard from her recently?
Przykładowe tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Przykładowe zdanie po angielsku: I heard that Michael is going to be a father.
Przykładowe tłumaczenie: [Słys

In [35]:
sampled_df = df.sample(n=2000, random_state=444).reset_index(drop=True)

for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']

    if is_english(text_to_translate, False) == False and is_english(text_to_translate, True) == False:
        sampled_df.at[index, f'dialog_prototype'] = text_to_translate
        continue

    translation = None
    i = 0

    while translation == None and i < 3:
        prompt = choose_and_format_promptv2(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)
        response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)
        translation = extract_translation(response)
        i+=1

    if translation == None:
        print(prompt)
        sampled_df.at[index, f'dialog_prototype'] = "ERROR"
        continue
        
  

    sampled_df.at[index, f'dialog_prototype'] = translation


    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/dialog_prototype_experimentsv8.csv", sep="|", index=False)

Processed 100 out of 2000 samples...
Processed 200 out of 2000 samples...
 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]


Oto kontekst: Poniżej znajduje się kawałek dialogu, rozmawia w nim Dwarf Doc (mężczyzna) z Geralt (mężczyzna)

1. Dwarf Doc - You, there! Outside the door! Geralt - that right?!
2. Geralt - Yeah. Geralt.


Na podstawie podanego dialogu, kontekstu oraz podanych wyżej przykładów, przetłumacz tylko i wyłącznie wypo